In [1]:
#导出500个tdb
import numpy as np
from pycalphad import Database
from espei.analysis import truncate_arrays
from espei.utils import database_symbols_to_fit, optimal_parameters

for i in range(2, 501):
    params = np.load('trace.npy')[:, -i:, :]
    lnprobor = np.load('lnprob.npy')[:, -i:]
    params, lnprobor = truncate_arrays(params, lnprobor)

    dbf = Database('fe-cr-ni.tdb')
    opt_params = dict(zip(database_symbols_to_fit(dbf), optimal_parameters(params, lnprobor)))
    dbf.symbols.update(opt_params)
    dbf.to_file(f'tdb/{i}.tdb')

In [2]:
import shutil
import re

for i in range(2, 501):
    shutil.copyfile('in.tdb', 'newtdb/{}.tdb'.format(i))
    with open('newtdb/{}.tdb'.format(i), 'r') as f1, open("tdb/{}.tdb".format(i), 'r') as f2:
        # 读取文件内容
        content1 = f1.read()
        content2 = f2.read()
        # 提取需要替换的字符串
        for j in range(3):
            start_str = f'FUNCTION VV000{j} 1 '
            end_str = '; 10000 N !'
            start_index1 = content1.index(start_str) + len(start_str)
            end_index1 = content1.index(end_str, start_index1)
            old_str = content1[start_index1:end_index1]
            start_index2 = content2.index(start_str) + len(start_str)
            end_index2 = content2.index(end_str, start_index2)
            new_str = content2[start_index2:end_index2]
            # 替换字符串
            content1 = content1.replace(old_str, new_str)
            # 写入新文件
            with open('newtdb/{}.tdb'.format(i), 'w') as f_new:
                f_new.write(content1)

In [ ]:
from pycalphad.core.utils import unpack_condition
from pycalphad.plot.utils import phase_legend
import pycalphad.variables as v
from matplotlib import collections as mc
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict
import numpy as np
from pycalphad import equilibrium
import pycalphad.variables as v
from pycalphad.plot.eqplot import eqplot
from pycalphad.core.utils import unpack_condition
from pycalphad.plot.utils import phase_legend
import pycalphad.variables as v
from matplotlib import collections as mc
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict
import numpy as np
from pycalphad import equilibrium
import pycalphad.variables as v
from pycalphad.plot.eqplot import eqplot
from pycalphad import Database

#dbf = Database('fe-ni-co_mcmc.tdb')
comps = ['FE', 'CR', 'NI', 'VA']
phases = ["BCC_A2","FCC_A1","SIGMA","FCC_L12","NI2CR"]
conds = {v.P: 101325, v.T: 1073, v.X('NI'): (0, 1, 0.05), v.X('FE'): (0, 1, 0.05)}
x=v.X('NI')
y=v.X('FE')
indep_comps = [key for key, value in conds.items() if isinstance(key, v.MoleFraction) and len(np.atleast_1d(value)) > 1]
indep_pots = [key for key, value in conds.items() if (type(key) is v.StateVariable) and len(np.atleast_1d(value)) > 1]
projection = 'triangular'
def plot_equilibrium(dbf, comps, phases, conds, ax):
    legend_generator=phase_legend
    eq = equilibrium(dbf, comps, phases, conds)
    phases = map(str, sorted(set(np.array(eq.Phase.values.ravel(), dtype='U')) - {''}, key=str))
    comps = map(str, sorted(np.array(eq.coords['component'].values, dtype='U'), key=str))
    eq['component'] = np.array(eq['component'], dtype='U')
    eq['Phase'].values = np.array(eq['Phase'].values, dtype='U')
    # Select all two- and three-phase regions
    two_phase_idx = np.nonzero(np.sum(eq.Phase.values != '', axis=-1, dtype=np.int_) == 2)
    legend_handles, colorlist = legend_generator(phases)
    found_two_phase = eq.Phase.values[two_phase_idx][..., :2]
    two_phase_x = eq.X.sel(component=x.species.name).values[two_phase_idx][..., :2]
    two_phase_y = eq.X.sel(component=y.species.name).values[two_phase_idx][..., :2]
    two_phase_plotcolors = np.array(list(map(lambda x: [colorlist[x[0]], colorlist[x[1]]], found_two_phase)), dtype='U')
    ax.scatter(two_phase_x[..., 0], two_phase_y[..., 0], s=5, c=two_phase_plotcolors[:, 0], edgecolors='None', zorder=2)
    ax.scatter(two_phase_x[..., 1], two_phase_y[..., 1], s=5, c=two_phase_plotcolors[:, 1], edgecolors='None', zorder=2)
# Create a new figure and axis object
fig, ax = plt.subplots(subplot_kw={'projection': projection},figsize=(8,6))
# Set the projection type
projection = 'triangular'
# Loop through the selected database files and plot the equilibrium phase diagram for each
for i in range(2, 501):
    dbf = Database(f'newtdb/{i}.tdb')
    plot_equilibrium(dbf, comps, phases, conds, ax)

# Add legend and labels
legend_handles, colorlist = phase_legend(phases)
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(handles=legend_handles, loc='center left', bbox_to_anchor=(1, 0.5), fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.grid(True)
plot_title = '-'.join([component.title() for component in sorted(comps) if component != 'VA'])
ax.set_title(plot_title, fontsize=20)
ax.set_xlabel('x (Ni)', fontsize=20)
ax.set_ylabel('x (Fe)', fontsize=20)
ax.yaxis.label.set_rotation(60)  # rotate ylabel
ax.yaxis.set_label_coords(x=0.15, y=0.48) 
# Show the plot
plt.savefig('1073allphase.jpg', bbox_inches = "tight", dpi=600)